# **Libararies**

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip install heartpy

In [ ]:
import heartpy as hp
import pandas as pd
import numpy as np

import bwr
import pywt
import statistics
import matplotlib.pyplot as plt
from scipy.signal import resample, find_peaks
from statsmodels.robust import mad

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

# **Functions**

In [ ]:
def waveletSmooth(x, wavelet="db4", level=1, title=None ):
    coeff = pywt.wavedec( x, wavelet, mode="per")
    sigma = mad( coeff[-level] )
    uthresh = sigma * np.sqrt( 2*np.log( len( x ) ) )
    coeff[1:] = ( pywt.threshold( i, value=uthresh, mode="soft" ) for i in coeff[1:] )
    y = pywt.waverec( coeff, wavelet, mode="per" )
    return y

In [ ]:
def dwt_extraction(ppg):
    # Preprocessing using Wavelet Smoothing & BWR   
    sinyal = waveletSmooth(ppg)
    _, ppg_out = bwr.bwr(sinyal)

    # Extraction using Multilevel 1D Discrete Wavelet Transform
    coeff = pywt.wavedec(ppg_out, 'sym3', level=4)
    return {'min': coeff[0].min(), 
            'max': coeff[0].max(), 
            'mean': statistics.mean(coeff[0]), 
            'med': statistics.median(coeff[0]), 
            'std': statistics.stdev(coeff[0])}

In [ ]:
def re_extraction(ppg, sample_rate):
    # Extraction using RE
    m = None
    try:
        denoise = hp.filter_signal(ppg, 
                                  cutoff = [0.8, 2.5], 
                                  filtertype = 'bandpass',
                                  sample_rate = sample_rate, 
                                  order = 3)
        resampled = resample(denoise, len(denoise) * 10)
        new_sample_rate = sample_rate * 10
        try:
            wd, m = hp.process(resampled, 
                                sample_rate=new_sample_rate, 
                                high_precision=True, 
                                clean_rr=False) 
        except: pass
    except:
        wd, m = hp.process(ppg,min,max,std, 
                            sample_rate=sample_rate, 
                            high_precision=True, 
                            clean_rr=False)
    return m 

In [ ]:
def rr_extraction(ppg, time):
    result = {'min': -100, 'max': -100, 'mean': -100, 'med': -100, 'std': -100}
    peaks, _ = find_peaks(ppg, distance=30)
    peaks_time = np.array([time[i] for i in peaks], dtype=np.datetime64)
    peaks_interval = np.diff(peaks_time).astype("float")/1000000
    if len(peaks_interval) >= 2:
        result['min'] = peaks_interval.min() 
        result['max'] = peaks_interval.max()
        result['mean'] = statistics.mean(peaks_interval) 
        result['med'] = statistics.median(peaks_interval) 
        result['std'] = statistics.stdev(peaks_interval)
    return result

# **Dataset**

In [ ]:
labels = [{'name': 'AF', 'dataset': 'AF1KOSONG.csv', 'loops': 19, 'sample_rate': 50, 'windows': 100}, 
          {'name': 'Normal', 'dataset': 'nclean.csv', 'loops': 19, 'sample_rate': 50, 'windows': 100},
          {'name': 'PVC', 'dataset': 'PVC1KOSONG.csv', 'loops': 12, 'sample_rate': 50, 'windows': 500}
         ]    

# **Extraction**

In [ ]:
hasil_dwt = []
hasil_re = []
hasil_rr = []
for label in labels:
    # Load ECG Signal
    df_data = pd.read_csv(label['dataset'], float_precision='round_trip')
    
    # Interval Segment Window
    for i in range(label['loops']):
        ppg = []
        time = []
        min = []
        max = []
        for j in range(label['windows']):
            ppg.append(df_data['PLETH'][(i*label['windows'])+j])
            time.append(pd.to_datetime(df_data['Time'][(i*label['windows'])+j]))

        if i not in [3, 6, 8, 10, 11, 14,15, 18]:
            # DWT Extraction
            features_dwt = dwt_extraction(ppg)
            features_dwt['label'] = label['name']
            hasil_dwt.append(features_dwt)

            # DYNAMIC Extraction
            features_re = re_extraction(ppg, sample_rate = label['sample_rate'])
            if features_re:
                features_re['label'] = label['name']
                hasil_re.append(features_re)

            # RR Extraction
            features_rr = rr_extraction(ppg, time)
            features_rr['label'] = label['name']
            hasil_rr.append(features_rr)

In [ ]:
le = LabelEncoder()
le.fit(['AF', 'Normal', 'PVC'])
le.classes_

array(['AF', 'Normal', 'PVC'], dtype='<U6')

## DWT Features

In [ ]:
result_dwt = pd.DataFrame(hasil_dwt)
result_dwt = result_dwt.sample(frac=1)
result_dwt = result_dwt.reset_index(drop=True)

result_dwt['class'] = le.transform(result_dwt['label'].values)
result_dwt

,min,max,mean,med,std,label,class
0,-247.920551,115.138561,-11.632843,49.231595,130.080619,AF,0
1,-189.251129,206.326931,-80.020658,-149.626793,141.859744,Normal,1
2,-1.102709,1.221279,0.037925,-0.020088,0.604476,PVC,2
3,-547.399020,251.216260,-218.044684,-231.251866,276.492531,Normal,1
4,-301.046905,135.955328,-0.478558,59.439387,146.166389,AF,0
5,-566.200859,256.851673,-37.007789,29.097550,251.652329,Normal,1
6,-268.899891,140.211201,-94.683306,-111.475790,127.688119,AF,0
7,-321.298400,147.541338,-3.420098,102.967335,194.887436,Normal,1
8,-386.068221,220.164904,-89.859620,-58.675604,169.432200,Normal,1
9,-346.461284,224.792217,-98.670221,-90.267803,158.070552,Normal,1


In [ ]:
result_dwt.corr()['class']
dfready = pd.concat([result_dwt]).reset_index(drop=True)
dfready.to_csv('dataDWT.csv')


## RE Features

In [ ]:
result_re = pd.DataFrame(hasil_re)
result_re = result_re.sample(frac=1)
result_re = result_re.reset_index(drop=True)
result_re = result_re.fillna(-1000)
result_re = result_re.drop(['sdsd', 'breathingrate'], axis=1)
result_re['class'] = le.transform(result_re['label'].values)

result_re

,bpm,ibi,sdnn,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,label,class
0,86.021505,697.500000,138.500000,277.000000,1.0,1.00,138.5,0.000000,0.000000,0.000000,-1000.000000,Normal,1
1,85.592011,701.000000,142.000000,284.000000,1.0,1.00,142.0,0.000000,0.000000,0.000000,-1000.000000,Normal,1
2,59.262511,1012.444444,134.395032,242.180202,1.0,1.00,185.0,170.386601,40.261644,21551.466075,4.231983,PVC,2
3,81.190798,739.000000,0.000000,-1000.000000,-1000.0,-1000.00,0.0,-1000.000000,-1000.000000,-1000.000000,-1000.000000,AF,0
4,88.105727,681.000000,119.000000,238.000000,1.0,1.00,119.0,0.000000,0.000000,0.000000,-1000.000000,Normal,1
5,122.199593,491.000000,270.000000,540.000000,1.0,1.00,270.0,0.000000,0.000000,0.000000,-1000.000000,AF,0
6,125.786164,477.000000,188.000000,376.000000,1.0,1.00,188.0,0.000000,0.000000,0.000000,-1000.000000,AF,0
7,60.483871,992.000000,113.812712,218.061631,1.0,1.00,109.0,153.992060,19.335646,9354.204889,7.964154,PVC,2
8,84.210526,712.500000,135.500000,271.000000,1.0,1.00,135.5,0.000000,0.000000,0.000000,-1000.000000,Normal,1
9,124.740125,481.000000,290.000000,580.000000,1.0,1.00,290.0,0.000000,0.000000,0.000000,-1000.000000,AF,0


In [ ]:
result_re.corr()['class']
dfready = pd.concat([result_re]).reset_index(drop=True)
dfready.to_csv('dataRE.csv')

## RR Features

In [ ]:
result_rr = pd.DataFrame(hasil_rr)
result_rr = result_rr.sample(frac=1)
result_rr = result_rr.reset_index(drop=True)

result_rr['class'] = le.transform(result_rr['label'].values)
result_rr

,min,max,mean,med,std,label,class
0,0.666,0.969,0.817500,0.8175,0.214253,Normal,1
1,-100.000,-100.000,-100.000000,-100.0000,-100.000000,AF,0
2,0.240,0.784,0.648000,0.7720,0.222164,PVC,2
3,-100.000,-100.000,-100.000000,-100.0000,-100.000000,Normal,1
4,0.765,0.787,0.776000,0.7760,0.015556,Normal,1
5,0.360,0.792,0.622400,0.6960,0.193279,PVC,2
6,0.779,0.835,0.807000,0.8070,0.039598,Normal,1
7,0.448,0.824,0.622667,0.5760,0.155078,PVC,2
8,-100.000,-100.000,-100.000000,-100.0000,-100.000000,AF,0
9,-100.000,-100.000,-100.000000,-100.0000,-100.000000,AF,0


In [ ]:
result_rr.corr()['class']
dfready = pd.concat([result_rr]).reset_index(drop=True)
dfready.to_csv('dataRR.csv')

In [ ]:
n_neighbors = 3

# **Combine**

In [ ]:
#Kombinasi 1 RR + DWT

dfready = pd.concat([result_rr,result_dwt]).reset_index(drop=True)
dfready.to_csv('dataRR+DWT.csv')

In [ ]:
#Kombinasi 2 RR + RE
dfready = pd.concat([result_rr,result_re]).reset_index(drop=True)
dfready.to_csv('dataRR+RE.csv')

In [ ]:
#Kombinasi 3 DWT + RE
dfready = pd.concat([result_re,result_dwt]).reset_index(drop=True)
dfready.to_csv('dataRE+DWT.csv')

In [ ]:
#Kombinasi RR + DWT + RE
dfready = pd.concat([result_rr,result_dwt,result_re]).reset_index(drop=True)
dfready.to_csv('dataKombinasi.csv')

# **Classification**

## DWT Classification

In [ ]:
X = result_dwt.drop(['label', 'class'], axis=1)
y = result_dwt['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, X_test.shape

((20, 5), (9, 5))

In [ ]:
knn_dwt = KNeighborsClassifier(n_neighbors=n_neighbors)
knn_dwt.fit(X_train, y_train)
y_pred = knn_dwt.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.60      1.00      0.75         3
           1       1.00      0.50      0.67         4
           2       1.00      1.00      1.00         2

    accuracy                           0.78         9
   macro avg       0.87      0.83      0.81         9
weighted avg       0.87      0.78      0.77         9



In [ ]:
cross_val_score(knn_dwt, X_test, y_test, cv=2)

array([0.4, 0. ])

## RE Classification

In [ ]:
X = result_re.drop(['pnn20', 'pnn50', 'sd1', 'sd2', 'label', 'class'], axis=1)
y = result_re['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, X_test.shape

((18, 7), (8, 7))

In [ ]:
knn_re = KNeighborsClassifier(n_neighbors=n_neighbors)
knn_re.fit(X_train, y_train)
y_pred = knn_re.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       0.67      0.67      0.67         3
           2       0.67      1.00      0.80         2

    accuracy                           0.75         8
   macro avg       0.78      0.78      0.76         8
weighted avg       0.79      0.75      0.75         8



In [ ]:
cross_val_score(knn_re, X_test, y_test, cv=2)

array([0.25, 0.25])

## RR Classification

In [ ]:
X = result_rr.drop(['label', 'class'], axis=1)
y = result_rr['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, X_test.shape

((20, 5), (9, 5))

In [ ]:
knn_rr = KNeighborsClassifier(n_neighbors=n_neighbors)
knn_rr.fit(X_train, y_train)
y_pred = knn_rr.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.67      1.00      0.80         2
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         6

    accuracy                           0.22         9
   macro avg       0.22      0.33      0.27         9
weighted avg       0.15      0.22      0.18         9



In [ ]:
cross_val_score(knn_rr, X_test, y_test, cv=2)

array([0.6, 1. ])

##COMBINATION CLASSIFICATION

In [ ]:
#Kombinasi 1 
result_rrdwt= pd.concat([result_rr,result_dwt]).reset_index(drop=True)
X = result_rrdwt.drop(['label', 'class'], axis=1)
y = result_rrdwt['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, X_test.shape

((40, 5), (18, 5))

In [ ]:
knn_rrdwt = KNeighborsClassifier(n_neighbors=n_neighbors)
knn_rrdwt.fit(X_train, y_train)
y_pred = knn_rrdwt.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.43      0.60         7
           1       0.40      1.00      0.57         4
           2       1.00      0.71      0.83         7

    accuracy                           0.67        18
   macro avg       0.80      0.71      0.67        18
weighted avg       0.87      0.67      0.68        18



In [ ]:
cross_val_score(knn_rrdwt, X_test, y_test, cv=2)

array([0.44444444, 0.66666667])

In [ ]:
#Kombinasi 2
result_rrre= pd.concat([result_rr,result_re]).reset_index(drop=True)
result_rrre
X = result_rrre.drop(['label','pnn20', 'pnn50', 'sd1', 'sd2', 'class'], axis=1)
y = result_rrre['class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, X_test.shape

((38, 12), (17, 12))

In [ ]:
knn_rrre = KNeighborsClassifier(n_neighbors=n_neighbors)
knn_rrre.fit(X_train, y_train)
y_pred = knn_rrre.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.43      0.60         7
           1       0.40      1.00      0.57         4
           2       1.00      0.71      0.83         7

    accuracy                           0.67        18
   macro avg       0.80      0.71      0.67        18
weighted avg       0.87      0.67      0.68        18

